In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
import cv2
from skimage import io
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications import ResNet101V2
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler, normalize
import os
import glob
import random

2024-04-20 16:03:18.783434: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-20 16:03:18.783894: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-20 16:03:18.788067: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-20 16:03:18.823864: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-20 16:03:20.541293

In [7]:
IMG_PATH = 'ADD THIS'

for CLASS in os.listdir(IMG_PATH):
    if not CLASS.startswith('.'):
        IMG_NUM = len(os.listdir(IMG_PATH + CLASS))
        for (n, FILE_NAME) in enumerate(os.listdir(IMG_PATH + CLASS)):
            img = IMG_PATH + CLASS + '/' + FILE_NAME
            if n < (0.2*IMG_NUM):
                shutil.copy(img, '../test/' + CLASS.upper() + '/' + FILE_NAME)
            else:
                shutil.copy(img, '../train/' + CLASS.upper() + '/' + FILE_NAME)

FileNotFoundError: [Errno 2] No such file or directory: 'ADD THIS'

In [ ]:
def plots(training_acc, testing_acc, training_loss, testing_loss):
    
    acc_categories = ['Training Accuracy', 'Testing Accuracy']
    acc_values = [training_acc, testing_acc]
    plt.figure(figsize=[12,6])
    
    plt.subplot(1, 3, 1)
    plt.bar(acc_categories, acc_values, width=0.2, color='green')
    plt.title('Training Vs Testing Accuracy')
    plt.ylabel('Accuracy')
    
    loss_categories = ['Training Loss', 'Testing Loss']
    loss_values = [training_loss, testing_loss]
    plt.subplot(1, 3, 2)
    plt.bar(loss_categories, loss_values, width=0.2)
    plt.title('Training Vs Testing Loss')
    plt.ylabel('Loss')
    plt.show()

In [ ]:
base_model = ResNet50V2(
    include_top=True, 
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
base_model.trainable = False

model = Sequential([
    base_model, 
    GlobalAveragePooling2D(),
    Flatten(),
    Dense(256, activation="relu", kernel_initializer='he_normal'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer=Adam(), 
               loss="binary_crossentropy", 
               metrics=['accuracy'])

model.summary()

checkpoint = tf.keras.callbacks.ModelCheckpoint('../models/resnet50v2_best.keras', 
                                                monitor='accuracy', verbose=1, 
                                                mode='max',save_best_only=True)
early = tf.keras.callbacks.EarlyStopping(monitor="accuracy", mode="max", restore_best_weights=True, patience=5)
callbacks_list = [checkpoint,early]

history = model.fit(train_ds, epochs=30, callbacks=callbacks_list)